In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from keras.layers import Activation, BatchNormalization, Concatenate, Dense, Embedding, Flatten, Reshape, Input, Multiply,Concatenate
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import Conv1D
from keras.models import Sequential, Model
from keras.optimizers import Adam

In [2]:
import math

In [3]:
from gensim.models import word2vec
import MeCab
import json
import hashlib
from googletrans import Translator
from keras.layers import Dropout
from keras.utils import np_utils
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import random
from gensim.models.doc2vec import Doc2Vec
from gensim.models.doc2vec import TaggedDocument
import torch
from transformers.modeling_bert import BertModel
from transformers.tokenization_bert_japanese import BertJapaneseTokenizer
%matplotlib inline

In [4]:
translator = Translator()
mt = MeCab.Tagger('-d /usr/local/lib/mecab/dic/mecab-ipadic-neologd/')
mt.parse('')
model_word = word2vec.Word2Vec.load("wiki_plus.model")

In [5]:
def get_tags(text):
    word = {}
    node = mt.parseToNode(text)
    while node:
        fields = node.feature.split(",")
        if (fields[0] == '名詞' or fields[0] == '動詞' or fields[0] == '形容詞') and node.surface in model_word.wv:
            w = node.surface
            word[w] = word.get(w, 0) + 1
        node = node.next
    return word

def weighted_mean_vec(text):
    v = np.zeros(model_word.vector_size)
    s = 1.0
    for w,weight in get_tags(text).items():
        v += weight * model_word.wv[w]  #Eventクラスeの単語wの個数＊単語wのベクトル
        s += weight
    return v / s

In [6]:
class Event:
    def __init__(self, id, type, score, desc, links):
        self.id = id
        self.type = type
        self.score = score
        self.desc = desc
        self.links = links

In [7]:
# JSON ファイルから event set をロード
def load_events(jsonfile):
    with open(jsonfile) as f:
        df = json.load(f)
    events = {x['id']: Event(x['id'], x['type'], x['score'], x['desc'], x['links']) for x in df} #eventsにidをkeyとしそのオブジェクトをvalueとした辞書を生成
    for k,x in events.items():
        x.links = [events[e] for e in x.links] #Event.linkの中身をidの配列からEventの配列に変更
    return events

In [8]:
events = load_events('sesaku2.json')

In [9]:
import pandas as pd
data = []
labels = []
columns=[]
index=[]
for k1, v1 in events.items():
    if v1.type[-1]=='部品':
        index.append(v1.desc)
for k1, v1 in events.items():
    if v1.type[-1]=='対策':
        if not v1.desc in columns:
            columns.append(v1.desc)
df = pd.DataFrame(index=index, columns=columns)
for k1, v1 in events.items():
    if v1.type[-1]=='部品':
        for k2, v2 in events.items():
            if v2.type[-1] == '対策':
                    if v2 in v1.links:
                        df.at[v1.desc, v2.desc] = 1
                    else:
                        df.at[v1.desc, v2.desc] = 0

In [10]:
df.shape

(173, 319)

In [11]:
taisaku_vec_word = {}
taisaku_count ={}
for i in df:
    taisaku_vec_word[i]=weighted_mean_vec(i)
    taisaku_count[i] = 0

#label →部品　生成→対策
data = []
buhin = []
taisaku = []
# data 
for index, row in df.iterrows():#部品
    x1 = weighted_mean_vec(index)
    buhin.append(x1)
    for i in df:#対策
        x2 =  taisaku_vec_word[i]#対策
        if row[i] ==1:
            data.extend([np.append(x1, x2)])
            taisaku_count[i] += 1
            
for k,v in taisaku_vec_word.items():
    taisaku.append(v)
data = np.array(data)
buhin = np.array(buhin)

In [12]:
#関係がある部品と対策の文書ベクトル
data = np.array(data)
data.shape

(7303, 400)

In [13]:
#部品の文書ベクトルだけ
buhin = np.array(buhin)
buhin.shape

(173, 200)

In [14]:
buhin.min()

-4.838822269439698

In [15]:
#対策の文書ベクトルだけ
taisaku = np.array(taisaku)
taisaku.shape

(319, 200)

In [16]:
taisaku_map = sorted(taisaku_count.items(),key=lambda x:x[1], reverse=True)

# GANの生成

In [17]:
#パラメータ
vec_length = 200
vec_shape =(vec_length,)
z_dim = 50

In [18]:
 #生成器
def build_generator(z_dim,vec_shape):
    
        model = Sequential()
        model.add(Dense(128,input_dim =( z_dim+vec_shape[0])))
        #model.add(BatchNormalization())
        model.add(Activation('relu'))
 
        model.add(Dense(200))
        model.add(Activation('tanh'))
        print('generator')
        model.summary()
        return model

In [19]:
#識別器
def build_discriminator(vec_shape):
    
    model = Sequential()
    
    model.add(Dense(128, input_dim=vec_shape[0]*2)) #activation
    model.add(Activation('relu'))
    
    #model.add(Dropout(0.2))
    model.add(Dense(128))
    model.add(Activation('relu'))
    #model.add(Dropout(0.2))
    model.add(Dense(1, activation='sigmoid'))
    print('discriminator')
    model.summary()
    return model

In [20]:
#generatorを学習する時のもの
def build_combined(generator,discriminator):
    z_y = Input(shape=(z_dim+vec_shape[0],)) #ランダムベクトル＋部品ベクトル
    vec_1 = Input(shape=vec_shape)              #部品ベクトル
    
    #ベクトルを生成
    gen_vec = generator(z_y)
    
    #discriminatorに入力するやつ
    vec_2 = Concatenate()([vec_1,gen_vec])  #[部品ベクトル,対策ベクトル]
    #discriminatorの重みとバイアスを固定
    discriminator.trainable = False
    
    valid = discriminator(vec_2)

    model = Model(inputs = [z_y, vec_1], outputs = valid)
    
    print('combined')
    model.summary()
    return model

In [21]:
#コンパイル
discriminator = build_discriminator(vec_shape)
discriminator.compile(loss='binary_crossentropy', optimizer=Adam(),metrics=['accuracy'])

generator = build_generator(z_dim,vec_shape)
    
combined = build_combined(generator,discriminator)
    
combined.compile(loss='binary_crossentropy',optimizer=Adam())

discriminator
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               51328     
_________________________________________________________________
activation_7 (Activation)    (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               16512     
_________________________________________________________________
activation_8 (Activation)    (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 129       
Total params: 67,969
Trainable params: 67,969
Non-trainable params: 0
_________________________________________________________________
generator
Model: "sequential_1"
_________________________________________________________________
Layer 

In [22]:
def data_scaling(data,change_min,change_max):
    max_data = math.ceil(data.max()) #実数より大きい整数
    min_data = math.floor(data.min()) #実数より小さい整数
    data_std = (data - min_data) / (max_data - min_data)
    data = data_std*(change_max - change_min) + change_min
    return data , min_data, max_data

In [23]:
losses = []
accuracies = []
iteration_checkpoints = []
generater_list = []
def train(generator_input, discriminator_input, iterations, batch_size, sample_interval):
    
    #generator_input = 部品ベクトル
    #discriminator_input = [部品ベクトル,対策ベクトル]
    #-1~1へスケーリング
    generator_input , _ , _ = data_scaling(generator_input,-1,1)
    discriminator_input , _ , _ =  data_scaling(discriminator_input,-1,1)
    #ラベル1
    real = np.ones((batch_size,1))
    #ラベル0
    fake = np.zeros((batch_size,1))
    generater_list = []
    
    for iteration in range(iterations):
        
        #-------------------
        #識別器の訓練
        #-------------------
        
        #本物のデータから適当に選ぶ
        idx = np.random.randint(0,len(discriminator_input),batch_size)
        #discriminatorの入力にいれる本物の[部品ベクトル,対策ベクトル]
        real_discriminator_input = discriminator_input[idx]
        
        #部品のベクトルの情報を含んだランダムベクトルを作る
        
        noize_y = np.random.randint(0,len(generator_input),batch_size)
        
        noize_y = generator_input[noize_y]
        
        #平均0標準偏差１の正規分布に従う乱数を生成
        noize_z = np.random.normal(0,1,(batch_size, z_dim))
        
        #ランダムベクトルと部品のベクトルを結合
        noize_z_y = np.concatenate((noize_z, noize_y),axis=1)
        
        #ジェネレータで偽の対策文書ベクトルを生成させる
        gen_vec = generator.predict(noize_z_y)
        
        #discriminatorの中にいれる偽の[部品ベクトル,対策ベクル]の生成
        fake_discriminator_input = np.concatenate((noize_y,gen_vec),axis=1)
        
        d_loss_real = discriminator.train_on_batch(real_discriminator_input, real)
        d_loss_fake = discriminator.train_on_batch(fake_discriminator_input, fake)
        
        #それぞれの損失関数を平均
        d_loss, accuracy = 0.5 * np.add(d_loss_real, d_loss_fake)
        
        #-------------------
        #生成器の訓練
        #-------------------
        #部品のベクトルの情報を含んだランダムベクトルを作る
        
        noize_y = np.random.randint(0,len(generator_input),batch_size)
        
        noize_y = generator_input[noize_y]

        #平均0標準偏差１の正規分布に従う乱数を生成
        noize_z = np.random.normal(0,1,(batch_size, z_dim))

        #ランダムベクトルと部品のベクトルを結合
        noize_z_y = np.concatenate((noize_z, noize_y),axis=1)

        #ジェネレータで偽の対策文書ベクトルを生成させる
        gen_vec = generator.predict(noize_z_y)
        
        g_loss = combined.train_on_batch([noize_z_y,noize_y],real)
        
        if (iteration +1) % sample_interval == 0:
            losses.append((d_loss, g_loss))
            accuracies.append(100.0 * accuracy)
            iteration_checkpoints.append(iteration +1)
            print("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" %(iteration +1,d_loss, 100.0*accuracy,g_loss))
            generater_list.append(generator)
    return generator, generater_list

In [31]:
iterations =10000
batch_size = 32
sample_interval = 1000

generator , generator_list = train(buhin, data , iterations, batch_size,sample_interval)

1000 [D loss: 0.047372, acc.: 98.44%] [G loss: 5.414024]
2000 [D loss: 0.016686, acc.: 100.00%] [G loss: 7.921521]
3000 [D loss: 0.072291, acc.: 95.31%] [G loss: 6.679328]
4000 [D loss: 0.036468, acc.: 98.44%] [G loss: 10.885368]
5000 [D loss: 0.016861, acc.: 100.00%] [G loss: 8.583508]
6000 [D loss: 0.000201, acc.: 100.00%] [G loss: 8.370418]
7000 [D loss: 0.157188, acc.: 96.88%] [G loss: 4.479087]
8000 [D loss: 0.009539, acc.: 100.00%] [G loss: 4.338871]
9000 [D loss: 0.009767, acc.: 100.00%] [G loss: 4.722936]
10000 [D loss: 0.077230, acc.: 98.44%] [G loss: 3.815525]


In [32]:
scaling_buhin ,min_data, max_data = data_scaling(buhin,-1,1)
noize_y = np.random.randint(0,len(scaling_buhin),1)
noize_y = buhin[noize_y]
noize_z = np.random.normal(0,1,(1, z_dim))
print(noize_y.shape)
noize_z_y = np.concatenate((noize_z, noize_y),axis=1)
print(noize_z_y.shape)
gen = np.array(generator(noize_z_y))

(1, 200)
(1, 250)


In [33]:
result , _,_ = data_scaling(gen, min_data,max_data)

In [34]:
result = result.reshape(200)

In [35]:
def cos_sim(v1, v2):
    return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))
class pycolor:
    RED = '\033[31m'
    BLUE = '\033[34m'
    END = '\033[0m'

In [36]:
def search_taisaku(text):
    vec = weighted_mean_vec(text)
    scaling_vec , _ , _ = data_scaling(vec,-1,1)
    scaling_vec = scaling_vec.reshape(1,200)
    noize_z = np.random.normal(0,1,(1, z_dim))
    noize = np.concatenate((noize_z, scaling_vec),axis=1)
    gen = np.array(generator(noize))
    result , _ , _ = data_scaling(gen, -5, 5) 
    score_sim = {}
    for k,v in taisaku_vec_word.items():
        score_sim[k]=cos_sim(result, v)
    score_sort = sorted(score_sim.items(),key=lambda x:x[1], reverse=True)
    for i_1, i_2 in score_sort[0:10]:
        print(i_1)
        print(i_2)

In [37]:
search_taisaku('afe')

製造部門だけでなく開発・設計などホワイトカラー業務にも適用できれいる。
[0.70772956]
メンテナンスコストやクレーム対応コストを加味した原価設計、歩留まり率設計を実施している
[0.707253]
原価企画・原価計画・原価維持・原価改善といった視点で、原価管理を実施している。
[0.70620052]
先行開発ゲートの定義は、通常開発の定義と異なっており、技術の棚という考え方がある
[0.68565887]
市場不具合を回収、分析、設計にフードバックする体制と組織がある
[0.67890323]
製品の開発・設計の段階から生産部門のスタッフが参画しており、開発から市場投入までのリードタイムを短縮している
[0.67247227]
開発～製造までの全社業務フロー図があり、連携を取りながら機能している
[0.6691743]
コンフィグレータが、製品構成、評価、生産まで考慮した仕様になっている。
[0.66738615]
生準業務マニュアルが整備。型・設備・治具・製作・整備技能が標準化されている
[0.66533257]
プランジャ当たり面の検査は、定量化・仕組み化・標準化しているか
[0.66473588]


In [38]:
buhin_name_list = []
for index, row in df.iterrows():#部品
    buhin_name_list.append(index)

In [39]:
def test(text):
    print('---------------部品----------------')
    print(text)
    print('---------------対策----------------')
    taisaku = []
    for index, row in df.iterrows():#部品
            if (index == text):
                for i in df:#対策
                    if row[i] ==1:
                        #print(i)
                        taisaku.append(i)
                break
    print('---------------予想----------------')
    vec = weighted_mean_vec(text)
    scaling_vec , _ , _ = data_scaling(vec,-1,1)
    scaling_vec = scaling_vec.reshape(1,200)
    noize_z = np.random.normal(0,1,(1, z_dim))
    noize = np.concatenate((noize_z, scaling_vec),axis=1)
    gen = np.array(generator(noize))
    result , _ , _ = data_scaling(gen, -5, 5)
    ####print(result)
    score_sim = {}
    for k,v in taisaku_vec_word.items():
        score_sim[k]=cos_sim(result, v)
    score_sort = sorted(score_sim.items(),key=lambda x:x[1], reverse=True)
    for i_1, i_2 in score_sort[0:10]:
        print(i_1)
        print(i_2)
        print(taisaku_count[i_1])
        if i_1 in taisaku:
            print(pycolor.BLUE+"ある"+pycolor.END)
        else:
            print(pycolor.RED+"なし"+pycolor.END)

In [40]:
text_1 = buhin_name_list[0]
test(text_1)

---------------部品----------------
エンジン搭載 吸気レゾネーター
---------------対策----------------
---------------予想----------------
定性的なアウトプットである個人の振り返りを実施。個人の成長・マネジメント軸を強化する。
[0.97267932]
152
ある
原価を維持・改善する活動を実施
[0.76019028]
21
なし
会社目標から製品、各機能組織ごとの、目標展開を実施。メンバー目標とも連携している
[0.69302565]
152
ある
職務拡充により作業者のモチベーション向上。品質を作りこむ意識が醸成。
[0.68430051]
152
ある
原価企画・原価計画・原価維持・原価改善といった視点で、原価管理を実施している。
[0.67269286]
152
ある
金型・治具の原価企画を実施。目標を設定して低減活動を実施
[0.66889759]
0
なし
社内のノウハウを標準化し、固有ノウハウを蓄積して効率的に活用
[0.65630081]
0
なし
ライン切り替え時の段取り作業の標準化、低減目標を設定し、年間活動計画で実施している
[0.65117009]
21
なし
物流コストを毎月把握し低減活動を実施。施策も見える化も実施。
[0.64844295]
0
なし
前機種の振り返りを実施。目標に対する達成度、課題と次の一手を適用している
[0.64817003]
21
なし


text_1 = buhin_name_list[10]
test(text_1)

In [41]:
text_1 = buhin_name_list[50]
test(text_1)

---------------部品----------------
エンジン本体 オイルパン　サブアセンブリ
---------------対策----------------
---------------予想----------------
定性的なアウトプットである個人の振り返りを実施。個人の成長・マネジメント軸を強化する。
[0.97267932]
152
なし
原価を維持・改善する活動を実施
[0.76019028]
21
ある
会社目標から製品、各機能組織ごとの、目標展開を実施。メンバー目標とも連携している
[0.69302565]
152
なし
職務拡充により作業者のモチベーション向上。品質を作りこむ意識が醸成。
[0.68430051]
152
なし
原価企画・原価計画・原価維持・原価改善といった視点で、原価管理を実施している。
[0.67269286]
152
なし
金型・治具の原価企画を実施。目標を設定して低減活動を実施
[0.66889759]
0
なし
社内のノウハウを標準化し、固有ノウハウを蓄積して効率的に活用
[0.65630081]
0
なし
ライン切り替え時の段取り作業の標準化、低減目標を設定し、年間活動計画で実施している
[0.65117009]
21
ある
物流コストを毎月把握し低減活動を実施。施策も見える化も実施。
[0.64844295]
0
なし
前機種の振り返りを実施。目標に対する達成度、課題と次の一手を適用している
[0.64817003]
21
ある


In [42]:
text_1 = buhin_name_list[100]
test(text_1)

---------------部品----------------
エンジン本体 クリーナーアセンブリ、エアW /エレメント
---------------対策----------------
---------------予想----------------
定性的なアウトプットである個人の振り返りを実施。個人の成長・マネジメント軸を強化する。
[0.97267932]
152
ある
原価を維持・改善する活動を実施
[0.76019028]
21
なし
会社目標から製品、各機能組織ごとの、目標展開を実施。メンバー目標とも連携している
[0.69302565]
152
ある
職務拡充により作業者のモチベーション向上。品質を作りこむ意識が醸成。
[0.68430051]
152
ある
原価企画・原価計画・原価維持・原価改善といった視点で、原価管理を実施している。
[0.67269286]
152
ある
金型・治具の原価企画を実施。目標を設定して低減活動を実施
[0.66889759]
0
なし
社内のノウハウを標準化し、固有ノウハウを蓄積して効率的に活用
[0.65630081]
0
なし
ライン切り替え時の段取り作業の標準化、低減目標を設定し、年間活動計画で実施している
[0.65117009]
21
なし
物流コストを毎月把握し低減活動を実施。施策も見える化も実施。
[0.64844295]
0
なし
前機種の振り返りを実施。目標に対する達成度、課題と次の一手を適用している
[0.64817003]
21
なし


##  これはモード崩壊（mode collapse）と呼ばれる現象で、generatorの学習に失敗して、訓練データの（しばしば多峰性の）分布全体を表現できずに訓練データの最頻値（mode）のみを学習してしまいます。全国民の期待に応える能力がなく、とりあえず多数派のための政策をつくる、みたいなイメージですかね。